In [1]:
using Erdos
using Distributions
using Gadfly
using Clustering
using MatrixNetworks

m = readdlm("Data/PowerUS/ElistPowerUS.txt");

const dim0 = size(m);
M_int = Array{Int64,2}(dim0[1],2);
for i =1:dim0[1]
	for j=1:2
		M_int[i,j] = Int64(m[i,j]);
	end
end	

N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]);

A0 =spzeros(N,N);
A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N);
A0 = A0+A0';
A,p = largest_component(A0);


########################### Free memory #################

A0=0;M_int=0;p=0; m=0;
gc();

In [2]:
d = sum(A,2);
s = sum(d);

r = 100;
N = length(d); 

v0 = rand(Uniform(-1,1),N,r);
v0 = v0./sqrt.(sum(abs2, v0, 1));
v0, t = qr(v0);
l0 = diag(t);


### Preprocess ###
u0 = rand(Uniform(-1,1),N,1);
u0 = u0./norm(u0);
n=0;
for it = 1:30
    u0 = A*u0 .- (d/s)*(d'*u0);
    n = norm(u0,2);
    u0 = u0/n;
end
shift = n;

l = l0;
##################

it_max=10000;
it = 1;
while it <=it_max
    v0 = A*v0 .- (d/s)*(d'*v0) .+ shift*v0;
    v0, t = qr(v0);
    l = diag(t);
    if norm(vec(abs.(l))-vec(abs.(l0)))<1e-03
             break;
    end
    l0 = l;
    it = it+1;
end

if it ==it_max
        print("No convergence")
end 

print(l0-shift)

[7.39025, 6.60827, 5.53959, 5.31819, 5.21386, 5.2483, 5.12896, 5.08091, 5.00709, 4.88611, 4.89964, 4.81005, 4.67227, 4.66711, 4.59583, 4.63066, 4.5427, 4.51668, 4.40028, 4.36442, 4.38951, 4.3521, 4.34703, 4.26999, 4.22953, 4.25015, 4.17729, 4.06584, 4.08556, 4.06666, 4.03017, 3.98073, 4.01737, 4.00309, 3.97926, 4.00143, 3.95047, 3.95822, 3.94929, 3.93676, 3.90151, 3.89346, 3.86802, 3.85728, 3.83261, 3.86804, 3.88023, 3.82603, 3.79671, 3.80133, 3.78725, 3.74604, 3.74558, 3.7517, 3.73077, 3.73149, 3.72371, 3.7039, 3.71612, 3.70477, 3.67049, 3.68038, 3.70329, 3.65076, 3.6566, 3.66659, 3.64342, 3.62552, 3.61284, 3.60885, 3.60806, 3.58278, 3.60762, 3.58912, 3.57891, 3.5097, 3.54157, 3.52501, 3.55023, 3.55133, 3.52104, 3.50864, 3.50765, 3.51436, 3.48791, 3.50064, 3.47676, 3.44916, 3.44769, 3.44766, 3.42715, 3.45356, 3.42704, 3.41946, 3.40921, 3.42829, 3.41968, 3.39132, 3.38426, 3.36197]

In [3]:
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")
include("Algo/Partition.jl")

n_clusters = 200;
n_it_Kmeans = 500;
n_updates = 100;
community = vec(zeros(N,1));
community0 = vec(zeros(N,1));

Q_best_best = 0;
n_c_best_best = 0;
n_updates_best_best = 0;
r_best = 2;
for r=2:100
    
    Sig = v0[:,1:r]';
    Q_best = 0;
    n_c_best = 0;
    n_c = 0;
    # Probability mass for sampling the initial centroids
    p = vec(d/s);
    n_updates_best = 0;

    for k =1:n_it_Kmeans

        community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
        n_c = length(unique(community0));
            if Q>Q_best
            community = community0;
            Q_best = Q;
            n_c_best = n_c;
            n_updates_best = n_updates;
            end
    end	

    community = Reorder(community);
    
    if Q_best>Q_best_best 
        r_best = r;
        Q_best_best = Q_best;
        n_c_best_best = n_c_best;
        n_updates_best_best = n_updates_best;
    end
    
end


	print("Number of updates: ")
	print(n_updates_best_best )
	print("\n")
	print("Number of communities: ")
	print(n_c_best_best )
	print("\n")

	print("Modularity: ")
	print(Q_best_best )
	print("\n")
	print("n: ")
	print(r_best )
	print("\n")


Number of updates: 0
Number of communities: 81
Modularity: 0.6702767973458469
n: 44
